# Data Collection for URP 353, Assignment 2

## Figure out what restaurant categories are available for the US market and save to a list

Category List Reference: https://www.yelp.com/developers/documentation/v3/all_category_list

In [44]:
import pandas as pd
import requests

In [45]:
import json
import time

In [46]:
categories = pd.read_json('https://www.yelp.com/developers/documentation/v3/all_category_list/categories.json')

In [47]:
categories.sample(1)

,alias,title,parents,country_whitelist,country_blacklist
910,motorcyclerepair,Motorcycle Repair,[auto],NaN,NaN


In [48]:
# categories.parents.astype(str).str.strip("['.']").unique()

In [49]:
categories['parents'] = categories['parents'].astype(str).str.strip("['.']")

In [50]:
categories.sample(1)

,alias,title,parents,country_whitelist,country_blacklist
1295,social_clubs,Social Clubs,arts,NaN,NaN


In [51]:
def available_US(white, black):
    if type(white) is float and type(black) is float:
        return True
    elif type(white) is float and 'US' in black:
        return False
    elif type(black) is float and not('US' in white):
        return False
    return True

In [52]:
US_filter = categories[['country_whitelist','country_blacklist']]\
    .apply(lambda x: available_US(x['country_whitelist'],x['country_blacklist']), axis = 1)

In [53]:
US_categories = categories[ 
    ((categories['parents']=='food') | (categories['parents']=='restaurants') )
    & US_filter
    ].iloc[:, 0:3]

In [54]:
US_categories.shape

(172, 3)

In [55]:
US_categories.sample(5)

,alias,title,parents
225,cafes,Cafes,restaurants
250,caribbean,Caribbean,restaurants
575,gelato,Gelato,food
1351,supperclubs,Supper Clubs,restaurants
361,customcakes,Custom Cakes,food


In [13]:
US_categories_li = US_categories.alias.to_list()

## Make a list of categories available in US that're culture / country related

In [14]:
US_cultural_categories_li = [x for x in US_categories_li if not (x in [
    'acaibowls',
    'asianfusion',
    'bagels', 'bakeries',
    'bbq',
    'beer_and_wine',
    'beverage_stores',
    'brasseries',
    'breakfast_brunch',
    'breweries',
    'bubbletea',
    'buffets',
    'burgers','butcher',
    'cafes',
    'cafeteria',
    'cakeshop',
    'cheesesteaks',
    'chicken_wings',
    'chickenshop',
    'chimneycakes',
    'cideries',
    'coffee',
    'coffeeroasteries',
    'comfortfood',
    'convenience',
    'creperies',
    'csa',
    'cupcakes',
    'customcakes',
    'delis',
    'desserts',
    'diners',
    'dinnertheater',
    'distilleries',
    'diyfood',
    'donuts',
    'empanadas',
    'farmersmarket',
    'fishnchips',
    'fondue',
    'food_court',
    'fooddeliveryservices',
    'foodstands',
    'foodtrucks',
    'gamemeat',
    'gastropubs',
    'gelato',
    'gluten_free',
    'gourmet',
    'greek',
    'grocery',
    'hkcafe',
    'honey',
    'hotdog',
    'hotdogs',
    'hotpot',
    'icecream',
    'importedfood',
    'indpak',
    'internetcafe',
    'intlgrocery',
    'juicebars',
    'kebab',
    'kombucha',
    'meaderies',
    'noodles',
    'organic_stores',
    'pizza',
    'poke',
    'popuprestaurants',
    'poutineries',
    'pretzels',
    'raw_food',
    'salad',
    'sandwiches',
    'seafood',
    'shavedice',
    'shavedsnow',
    'smokehouse',
    'soulfood',
    'soup',
    'southern',
    'steak',
    'streetvendors',
    'supperclubs',
    'sushi',
    'tapas',
    'tapasmallplates',
    'tea',
    'tex-mex',
    'vegan',
    'vegetarian',
    'waffles',
    'waterstores',
    'wraps'
])]

In [15]:
US_cultural_categories_li

['afghani',
 'african',
 'arabian',
 'argentine',
 'armenian',
 'australian',
 'austrian',
 'bangladeshi',
 'basque',
 'belgian',
 'brazilian',
 'british',
 'bulgarian',
 'burmese',
 'cajun',
 'cambodian',
 'caribbean',
 'catalan',
 'chinese',
 'cuban',
 'czech',
 'eritrean',
 'ethiopian',
 'filipino',
 'french',
 'georgian',
 'german',
 'guamanian',
 'halal',
 'hawaiian',
 'himalayan',
 'honduran',
 'hungarian',
 'iberian',
 'indonesian',
 'irish',
 'italian',
 'japanese',
 'korean',
 'kosher',
 'laotian',
 'latin',
 'malaysian',
 'mediterranean',
 'mexican',
 'mideastern',
 'modern_european',
 'mongolian',
 'moroccan',
 'newamerican',
 'newmexican',
 'nicaraguan',
 'pakistani',
 'panasian',
 'persian',
 'peruvian',
 'piadina',
 'polish',
 'polynesian',
 'portuguese',
 'russian',
 'scandinavian',
 'scottish',
 'singaporean',
 'slovakian',
 'somali',
 'spanish',
 'srilankan',
 'syrian',
 'taiwanese',
 'thai',
 'tradamerican',
 'turkish',
 'ukrainian',
 'uzbek',
 'vietnamese']

In [16]:
len(US_cultural_categories_li)

76

## Identify a list of U.S. cities whose food diversity we're interested in

In [17]:
locales = ['en_US']

Read in a list of U.S. state capitals

In [18]:
state_capitals_df = pd.read_json('https://raw.githubusercontent.com/vega/vega/master/docs/data/us-state-capitals.json', 
                              orient = 'records')

In [19]:
state_capitals = state_capitals_df[['city','state']]
state_capitals['query'] = (state_capitals.city + ', ' + state_capitals.state)

<ipython-input-19-d222befebcf3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_capitals['query'] = (state_capitals.city + ', ' + state_capitals.state)


Read in a list of major US cities with top 50 populations

In [20]:
major_cities_df = pd.read_json('https://gist.githubusercontent.com/Miserlou/c5cd8364bf9b2420bb29/raw/2bf258763cdddd704f8ffd3ea9a3e81d25e2c6f6/cities.json', 
                              orient = 'records')

In [21]:
major_cities = major_cities_df[ major_cities_df['rank'] <= 50 ][['city','state']]
major_cities['query'] = (major_cities.city + ', ' + major_cities.state)


Join state_capitals with major cities (later changed to top 50 US cities plus Ann Arbor due to rate limit)

In [22]:
# locations_df = pd.concat([state_capitals, major_cities]).drop_duplicates().reset_index(drop=True)
locations_df = major_cities
locations_df.loc[51] =  ['Ann Arbor', 'Michigan', 'Ann Arbor, Michigan'] 

In [23]:
locations_df.head()

,city,state,query
0,New York,New York,"New York, New York"
1,Los Angeles,California,"Los Angeles, California"
2,Chicago,Illinois,"Chicago, Illinois"
3,Houston,Texas,"Houston, Texas"
4,Philadelphia,Pennsylvania,"Philadelphia, Pennsylvania"


In [24]:
locations_df.shape

(51, 3)

Get list of query locations

In [25]:
locations = locations_df['query'].to_list()

## Query Yelp API for count of businesses for each cultural/national cuisine in each location

### Make and write request functions for retriving count of businesses

In [26]:
def get_search_parameters(city, category):
    params = {}
    params['location'] = city
    params['categories'] = category
    params['limit'] = 50
    return params


In [27]:
def get_results(params):
    api_key = 'a-I1scMVVk1RS_zHPpH59SYDzWY59sGAI5LcK5nT8XIaW30lf_WshH6MSE2AKeT1zdljEaiTKF8pjKawh1KL2Jys5mRD8yJt3IIvyd0gu0oUNbJpGiIgvBiwH_JvX3Yx'
    headers = {'Authorization': 'Bearer %s' % api_key}
    url = "https://api.yelp.com/v3/businesses/search"
    
    response = requests.get(url, params=params, headers=headers)
    time.sleep(0.5)
    
    if response.status_code == 200:
        response = json.loads(response.text)
        return response
    else:
        print ('Error code %s' % response.status_code)
        print(json.loads(response.text))
        return 'Error code %s' % response.status_code

Test connection

In [28]:
test_response = get_results(get_search_parameters('NYC', 'chinese'))

In [29]:
def read_response(city, category, res):
    d = {}
    d['query'] = city
    d['category'] = category
    
    try:
        d['total_business_count'] = res['total']
        if res['total'] != 0:
            sample = pd.DataFrame(res['businesses'])
            d['sample_rating'] = sample['rating'].mean()
            d['sample_review_count'] = sample['review_count'].mean()
        else: 
            d['total_business_count'] = 0
            d['sample_rating'] = 'NA'
            d['sample_review_count'] = 'NA'
    except:
        d['total_business_count'] = 0
        d['sample_rating'] = 'Error'
        d['sample_review_count'] = 'Error'
        
    return d

In [30]:
read_response('NYC', 'chinese', test_response)

{'query': 'NYC',
 'category': 'chinese',
 'total_business_count': 1700,
 'sample_rating': 4.1,
 'sample_review_count': 1196.06}

### Literatively retreive count of businesses for each of the 76 cultural/national cuisine

Caching

In [31]:
output_path = 'food_diversity.json'

In [32]:
def get_counts(fname):
    try:
        cache_file = open(output_path,'r')
        cache_contents = cache_file.read()
        cache_file.close()
        cache = json.loads(cache_contents)
    except:
        cache = []
        
    if len(cache) > 0:
        cached_cities = [ (r['query'],r['category']) for r in cache]
    else:
        cached_cities = []

    for city in locations:
        for category in US_cultural_categories_li:
            if (city,category) in cached_cities:
                print ( "%s's category %s already formatted" % (city,category))
                continue
            params = get_search_parameters(city, category)
            response = get_results(params)
            cache.append( read_response(city, category, response) )
        cacheFile = open(fname, "w")
        json.dump(cache, cacheFile)
        cacheFile.close()


    return cache
        

In [33]:
counts = get_counts(output_path)

Error code 500
{'error': {'code': 'INTERNAL_ERROR', 'description': 'Something went wrong internally, please try again later.'}}
Error code 500
{'error': {'code': 'INTERNAL_ERROR', 'description': 'Something went wrong internally, please try again later.'}}
Error code 500
{'error': {'code': 'INTERNAL_ERROR', 'description': 'Something went wrong internally, please try again later.'}}
Error code 500
{'error': {'code': 'INTERNAL_ERROR', 'description': 'Something went wrong internally, please try again later.'}}
Error code 500
{'error': {'code': 'INTERNAL_ERROR', 'description': 'Something went wrong internally, please try again later.'}}
Error code 500
{'error': {'code': 'INTERNAL_ERROR', 'description': 'Something went wrong internally, please try again later.'}}
Error code 500
{'error': {'code': 'INTERNAL_ERROR', 'description': 'Something went wrong internally, please try again later.'}}
Error code 500
{'error': {'code': 'INTERNAL_ERROR', 'description': 'Something went wrong internally, ple

## Overview of the retreived data

In [34]:
food_diversity = pd.DataFrame(counts)

In [35]:
food_diversity.shape

(3876, 5)

In [37]:
food_diversity.sample(5)

,query,category,total_business_count,sample_rating,sample_review_count
2998,"Atlanta, Georgia",indonesian,3,4.5,74.3333
1362,"Detroit, Michigan",thai,22,3.77273,102.727
3729,"Arlington, Texas",australian,1,5,313
2713,"Long Beach, California",panasian,17,3.79412,330.118
1339,"Detroit, Michigan",mongolian,5,3.2,125


In [40]:
food_diversity.head(5)

,query,category,total_business_count,sample_rating,sample_review_count
0,"New York, New York",afghani,18,3.75,150.111
1,"New York, New York",african,48,4.11458,115.646
2,"New York, New York",arabian,5,3.9,51.6
3,"New York, New York",argentine,21,3.90476,182.857
4,"New York, New York",armenian,5,4.5,27.4
